In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

# 모델과 토크나이저 로드
def load_model_and_tokenizer(model_path):
    model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model.eval()
    model.config.pad_token_id = tokenizer.pad_token_id
    return tokenizer, model

# 모델의 응답에서 <|start_header_id|>assistant<|end_header_id|> 이후의 답변만 추출
def extract_model_response(result, tokenizer):
    decoded_result = tokenizer.decode(result, skip_special_tokens=True)
    assistant_start = "assistant"
    if assistant_start in decoded_result:
        return decoded_result.split(assistant_start)[-1].strip()
    else:
        return decoded_result.strip()

# 번역된 문장만 출력하는 함수 (KR_Meaning -> KR_Idiom)
def find_korean_idiom(meaning, tokenizer, model):
    system_prompt = "You are an idiom expert. Your goal is to identify the idiom based on the meaning provided by the user."
    user_input = f"What is the idiom that means '{meaning}'? Please provide only the idiom without any additional explanation."
    
    prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\n{user_input}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>"
    
    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    output_tokens = model.generate(**inputs, max_length=100, repetition_penalty=1.5, do_sample=True, top_p=0.92)
    
    result = extract_model_response(output_tokens[0], tokenizer)  # 답변만 추출
    print(f"KR_Meaning: {meaning}\nFound Idiom: {result}\n")
    return result.strip()

# 번역된 문장만 출력하는 함수 (KR -> EN)
def translate_kr_to_en(sentence, tokenizer, model):
    system_prompt = "You are a professional translator proficient in Korean and English. Please translate the given Korean sentence into English accurately, and only provide the translated sentence."
    user_input = f"Translate the following sentence into English: {sentence}"
    
    prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\n{user_input}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>"
    
    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    output_tokens = model.generate(**inputs, max_length=100, repetition_penalty=1.5, do_sample=True, top_p=0.92)
    
    result = extract_model_response(output_tokens[0], tokenizer)
    print(f"KR_Sentence: {sentence}\nTranslated to English: {result}\n")
    return result.strip()

# 번역된 문장만 출력하는 함수 (EN -> KR)
def translate_en_to_kr(sentence, tokenizer, model):
    system_prompt = "You are a professional translator proficient in Korean and English. Please translate the given English sentence into Korean accurately, and only provide the translated sentence."
    user_input = f"Translate the following sentence into Korean: {sentence}"
    
    prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\n{user_input}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>"
    
    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    output_tokens = model.generate(**inputs, max_length=100, repetition_penalty=1.5, do_sample=True, top_p=0.92)
    
    result = extract_model_response(output_tokens[0], tokenizer)
    print(f"EN_Sentence: {sentence}\nTranslated to Korean: {result}\n")
    return result.strip()

# KR 문장을 EN으로 번역하는 instruction 추가
def translate_kr_to_en_inst(sentence, tokenizer, model):
    prompt = f"Find and translate the idiom in the following Korean sentence into their corresponding English idioms, while translating the rest of the sentence into natural English. Provide only the translated sentence.\n\n# Korean Sentence:\n{sentence}\n\n# Translation:\n"
    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    output_tokens = model.generate(**inputs, max_length=100, repetition_penalty=1.5, do_sample=True, top_p=0.92)
    
    result = extract_model_response(output_tokens[0], tokenizer)
    print(f"KR_Sentence: {sentence}\nTranslated to English (Instruction): {result}\n")
    return result.strip()

# EN 문장을 KR으로 번역하는 instruction 추가
def translate_en_to_kr_inst(sentence, tokenizer, model):
    prompt = f"Find and translate the idiom in the following English sentence into their corresponding Korean idioms, while translating the rest of the sentence into natural Korean. Provide only the translated sentence.\n\n# English Sentence:\n{sentence}\n\n# Translation:\n"
    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    output_tokens = model.generate(**inputs, max_length=100, repetition_penalty=1.5, do_sample=True, top_p=0.92)
    
    result = extract_model_response(output_tokens[0], tokenizer)
    print(f"EN_Sentence: {sentence}\nTranslated to Korean (Instruction): {result}\n")
    return result.strip()

# CSV 데이터 로드 함수
def load_data(input_csv_path):
    df = pd.read_csv(input_csv_path)
    return df

# 추론 작업 수행 및 모든 Task 결과 저장
def perform_inference_and_save_separately(model_path, input_csv_path, output_csv_paths):
    tokenizer, model = load_model_and_tokenizer(model_path)
    df = load_data(input_csv_path)
    
    df_idiom = df.copy()
    df_idiom['Found_KR_Idiom'] = ''
    df_kr_to_en = df.copy()
    df_kr_to_en['Translated_English'] = ''
    df_en_to_kr = df.copy()
    df_en_to_kr['Translated_Korean'] = ''
    df_kr_to_en_inst = df.copy()
    df_kr_to_en_inst['Translated_English_inst'] = ''
    df_en_to_kr_inst = df.copy()
    df_en_to_kr_inst['Translated_Korean_inst'] = ''

    for idx, row in df.iterrows():
        kr_meaning = row['KR_Meaning']
        kr_sentence = row['KR_Sentence']
        en_sentence = row['Sentence']
        
        # 1. Idiom 찾기 (KR_Meaning -> KR_Idiom)
        found_idiom = find_korean_idiom(kr_meaning, tokenizer, model)
        df_idiom.at[idx, 'Found_KR_Idiom'] = found_idiom
        
        # 2. KR -> EN 번역
        translated_kr_en = translate_kr_to_en(kr_sentence, tokenizer, model)
        df_kr_to_en.at[idx, 'Translated_English'] = translated_kr_en
        
        # 3. EN -> KR 번역
        translated_en_kr = translate_en_to_kr(en_sentence, tokenizer, model)
        df_en_to_kr.at[idx, 'Translated_Korean'] = translated_en_kr
        
        # 4. Instruction 기반 KR -> EN 번역
        translated_kr_en_inst = translate_kr_to_en_inst(kr_sentence, tokenizer, model)
        df_kr_to_en_inst.at[idx, 'Translated_English_inst'] = translated_kr_en_inst
        
        # 5. Instruction 기반 EN -> KR 번역
        translated_en_kr_inst = translate_en_to_kr_inst(en_sentence, tokenizer, model)
        df_en_to_kr_inst.at[idx, 'Translated_Korean_inst'] = translated_en_kr_inst

    # CSV 저장
    df_idiom[['KR_Meaning', 'KR_Idiom', 'Found_KR_Idiom']].to_csv(output_csv_paths['idiom'], index=False)
    df_kr_to_en[['KR_Sentence', 'Idiom', 'Translated_English']].to_csv(output_csv_paths['kr_to_en'], index=False)
    df_en_to_kr[['Sentence', 'KR_Idiom', 'Translated_Korean']].to_csv(output_csv_paths['en_to_kr'], index=False)
    df_kr_to_en_inst[['KR_Sentence', 'Idiom', 'Translated_English_inst']].to_csv(output_csv_paths['kr_to_en_inst'], index=False)
    df_en_to_kr_inst[['Sentence', 'KR_Idiom', 'Translated_Korean_inst']].to_csv(output_csv_paths['en_to_kr_inst'], index=False)

    print("!!SAVED!!")

# CSV 데이터 경로 및 모델 경로
#model_path = "/data/uijih/8b_instruct/saveded_instruct-full-1"
model_path = "meta-llama/Meta-Llama-3.1-8B-Instruct"
input_csv_path = '/data/uijih/Seed50_for_Parallel_Dataset_ENKR_idiomKB_0.8_example.csv'

output_csv_paths = {
    'idiom': './results_mean_to_idiom-1.csv',
    'kr_to_en': './results_kr_to_en-1.csv',
    'en_to_kr': './results_en_to_kr-1.csv',
    'kr_to_en_inst': './results_kr_to_en-inst-1.csv',
    'en_to_kr_inst': './results_en_to_kr-inst-1.csv'
}

perform_inference_and_save_separately(model_path, input_csv_path, output_csv_paths)


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

# 모델과 토크나이저 로드
def load_model_and_tokenizer(model_path):
    model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model.eval()
    #model.config.pad_token_id = tokenizer.pad_token_id
    return tokenizer, model

# 프롬프트 템플릿을 기반으로 한 결과 추출
def generate_prompt_and_infer(prompt_template, user_input, tokenizer, model, max_length=100):
    prompt = prompt_template.format(**user_input)
    
    # 프롬프트를 토큰화하고 모델에 입력
    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    output_tokens = model.generate(**inputs, max_length=max_length, repetition_penalty=1.5, do_sample=True, top_p=0.92)

    # 모델의 출력 결과를 디코딩
    result = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    return result.strip()

# 번역된 문장만 출력하는 함수 (KR_Meaning -> KR_Idiom)
def find_korean_idiom(kr_meaning, tokenizer, model, prompt_template):
    user_input = {'meaning': kr_meaning}
    result = generate_prompt_and_infer(prompt_template, user_input, tokenizer, model)
    print(f"KR_Meaning: {kr_meaning}\nFound Idiom: {result}\n")
    return result

# 번역된 문장만 출력하는 함수 (KR -> EN)
def translate_kr_to_en(sentence, tokenizer, model, prompt_template):
    user_input = {'source': sentence}
    result = generate_prompt_and_infer(prompt_template, user_input, tokenizer, model)
    print(f"KR_Sentence: {sentence}\nTranslated to English: {result}\n")
    return result

# 번역된 문장만 출력하는 함수 (EN -> KR)
def translate_en_to_kr(sentence, tokenizer, model, prompt_template):
    user_input = {'source': sentence}
    result = generate_prompt_and_infer(prompt_template, user_input, tokenizer, model)
    print(f"EN_Sentence: {sentence}\nTranslated to Korean: {result}\n")
    return result

# CSV 데이터 로드 함수
def load_data(input_csv_path):
    df = pd.read_csv(input_csv_path)
    return df

# 추론 작업 수행 및 모든 Task 결과 저장
def perform_inference_and_save_separately(model_path, input_csv_path, output_csv_paths, prompt_templates):
    tokenizer, model = load_model_and_tokenizer(model_path)
    df = load_data(input_csv_path)
    
    df_idiom = df.copy()
    df_idiom['Found_KR_Idiom'] = ''
    df_kr_to_en = df.copy()
    df_kr_to_en['Translated_English'] = ''
    df_en_to_kr = df.copy()
    df_en_to_kr['Translated_Korean'] = ''

    for idx, row in df.iterrows():
        kr_meaning = row['KR_Meaning']
        kr_sentence = row['KR_Sentence']
        en_sentence = row['Sentence']
        
        # 1. Idiom 찾기 (KR_Meaning -> KR_Idiom)
        found_idiom = find_korean_idiom(kr_meaning, tokenizer, model, prompt_templates['kr_reverse'])
        df_idiom.at[idx, 'Found_KR_Idiom'] = found_idiom
        
        # 2. KR -> EN 번역
        translated_kr_en = translate_kr_to_en(kr_sentence, tokenizer, model, prompt_templates['kr_to_en'])
        df_kr_to_en.at[idx, 'Translated_English'] = translated_kr_en
        
        # 3. EN -> KR 번역
        translated_en_kr = translate_en_to_kr(en_sentence, tokenizer, model, prompt_templates['en_to_kr'])
        df_en_to_kr.at[idx, 'Translated_Korean'] = translated_en_kr

    # CSV 저장
    df_idiom[['KR_Meaning', 'KR_Idiom', 'Found_KR_Idiom']].to_csv(output_csv_paths['idiom'], index=False)
    df_kr_to_en[['KR_Sentence', 'Idiom', 'Translated_English']].to_csv(output_csv_paths['kr_to_en'], index=False)
    df_en_to_kr[['Sentence', 'KR_Idiom', 'Translated_Korean']].to_csv(output_csv_paths['en_to_kr'], index=False)

    print("!!SAVED!!")

# CSV 데이터 경로 및 모델 경로
model_path = "/data/uijih/8b/saved_full_finetuned-1"
input_csv_path = '/data/uijih/Seed50_for_Parallel_Dataset_ENKR_idiomKB_0.8_example.csv'

output_csv_paths = {
    'idiom': './results_mean_to_idiom.csv',
    'kr_to_en': './results_kr_to_en.csv',
    'en_to_kr': './results_en_to_kr.csv'
}

# 프롬프트 템플릿 설정
prompt_templates = {
    'kr_reverse': "다음 뜻을 가진 관용구를 찾아보세요: '{meaning}'.",
    'kr_to_en': "'{source}'라는 문장을 한국어에서 영어로 번역해 주세요.",
    'en_to_kr': "Translate the following sentence into Korean: '{source}'"
}

perform_inference_and_save_separately(model_path, input_csv_path, output_csv_paths, prompt_templates)


/data/uijih/anaconda3/envs/python3.11.3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:27<00:00,  6.78s/it]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


KR_Meaning:  아주 하기 쉬운 일을 비유적으로 이르는 말.
Found Idiom: 다음 뜻을 가진 관용구를 찾아보세요:'아주 하기 쉬운 일을 비유적으로 이르는 말.'. 

    # 의미:
     매우 쉽다는말.

    #관용 구문 :
      겉옷만 차리도록

        #해석::
         사물을 이해하지 못함을 일화로 표현하다.



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


KR_Sentence:  겉보리 돈 사기가 수양딸로 며느리 삼기보다 쉽다고 이 정도 일이면 만족하고 해야지.
Translated to English: ' 겉보리 돈 사기가 수양딸로 며느리 삼기보다 쉽다고 이 정도 일이면 만족하고 해야지.'라는 문장을 한국어에서 영어로 번역해 주세요. 

    #한국어 문장:
     외 얽고 벽 친다

    #번역:
      to be in deep trouble or difficulty

EN_Sentence:  The exam was a piece of cake
Translated to Korean: Translate the following sentence into Korean:'The exam was a piece of cake'



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

KR_Sentence:  비싼 돈 주고 형편없는 대접을 받은 손님은 화가 머리끝까지 나고야 말았다.
Translated to English: ' 비싼 돈 주고 형편없는 대접을 받은 손님은 화가 머리끝까지 나고야 말았다.'라는 문장을 한국어에서 영어로 번역해 주세요. 

    #한국어 문장:
     천한 재료에 높은 가격을 붙이는 경우를 이르는말.

    #번역:
     to charge exorbitant prices for poor quality goods or services

EN_Sentence:  My father will blow his top when he sees what happened to the car.
Translated to Korean: Translate the following sentence into Korean:'My father will blow his top when he sees what happened to the car.'



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


KR_Meaning: ‘죽다’를 비유적으로 이르는 말.
Found Idiom: 다음 뜻을 가진 관용구를 찾아보세요: '‘죽다’를 비유적으로 이르는 말.'. 

    # 의미:
     아주 하기 쉬운 일을 비유적으로 일컬어함.

    # 관용 구문 :
      겉박이 되도록 한다.



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


KR_Sentence:  그렇게 젋은 목숨들이 한 줌 재가 되는 안타까운 사건이 있다.
Translated to English: ' 그렇게 젋은 목숨들이 한 줌 재가 되는 안타까운 사건이 있다.'라는 문장을 한국어에서 영어로 번역해 주세요. 

    # 번역:
     참으로 슬프고 깊은 마음을 받아들여야 할 일이 일어난다.

    # 출처:")
        https://ko.dict.naver.com/#/en-kr/Junior%20memberships

EN_Sentence:  Her eyes fluttered open for a moment and then she breathed her last.
Translated to Korean: Translate the following sentence into Korean:'Her eyes fluttered open for a moment and then she breathed her last.'



In [ ]:
print(model.config.pad_token_id)